**Fachprojekt Dokumentenanalyse** *SS24* -- *Arthur Matei, Gernot A. Fink* -- *Technische Universität Dortmund, Lehrstuhl XII, Mustererkennung*
---
# Aufgabe 2: Bag-of-Words, Klassifikation

In dieser Aufgabe sollen unbekannte Dokumente zu bekannten Kategorien automatisch zugeordnet werden.

Die dabei erforderlichen numerischen Berechnungen lassen sich im Vergleich zu einer direkten Implementierung in Python erheblich einfacher mit NumPy / SciPy durchfuehren. Die folgende Aufgabe soll Ihnen die Unterschiede anhand eines kleinen Beispiels verdeutlichen.

Geben Sie fuer jede Katgorie des Brown Corpus die durchschnittliche Anzahl von Woertern pro Dokument aus. Bestimmen Sie auch die Standardabweichung. Stellen Sie diese Statistik mit einem bar plot dar. Verwenden Sie dabei auch Fehlerbalken (siehe visualization.hbar_plot)

Berechnen Sie Mittelwert und Standardabweichung jeweils:

 - nur mit Python Funktion
   hilfreiche Funktionen: sum, float, math.sqrt, math.pow

 - mit NumPy
   hilfreiche Funktionen: np.array, np.mean, np.std

http://docs.python.org/3/library/math.html
http://docs.scipy.org/doc/numpy/reference/generated/numpy.mean.html
http://docs.scipy.org/doc/numpy/reference/generated/numpy.std.html

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import sys

if ".." not in sys.path:
    sys.path.append("..")

import math
import numpy as np
from common.corpus import CorpusLoader
from common.visualization import hbar_plot

# Laden des Brown Corpus
CorpusLoader.load()
brown = CorpusLoader.brown_corpus()
brown_categories = brown.categories()



ModuleNotFoundError: No module named 'ipympl'

 ## Klassifikation von Dokumenten

Nachdem Sie sich nun mit der Struktur und den Eigenschaften des Brown Corpus vertraut gemacht haben, soll er die Datengrundlage fuer die Evaluierung von Algorithmen zur automatischen Klassifikation von Dokumenten bilden.
In der Regel bestehen diese Algorithmen aus drei Schritten:
 - Vorverarbeitung
 - Merkmalsberechnung
 - Klassifikation

Bei der Anwendung auf Dokumente (Texte) werden diese Schritte wie folgt umgesetzt:

 - **Vorverarbeitung:** Filterung von stopwords und Abbildung von Woertern auf Wortstaemme.
 - **Merkmalsberechnung:** Jedes Dokument wird numerisch durch einen Vektor repraesentiert (--> NumPy), der moeglichst die bzgl. der Klassifikation bedeutungsunterscheidenden Informationen enthaehlt.
 - **Klassifikation:** Jedem Merkmalsvektor (Dokument) wird ein Klassenindex (Kategorie) zugeordnet.

Details finden Sie zum Beispiel in:
http://www5.informatik.uni-erlangen.de/fileadmin/Persons/NiemannHeinrich/klassifikation-von-mustern/m00-www.pdf (section 1.3)

Eine sehr verbreitete Merkmalsrepraesentation fuer (textuelle) Dokumente sind sogenannte Bag-of-Words. Dabei wird jedes Dokument durch ein Histogram (Verteilung) ueber Wortfrequenzen repraesentiert. Man betrachtet dazu das Vorkommen von 'typischen' Woertern, die durch ein Vokabular gegeben sind.

Bestimmen Sie ein Vokabular, also die typischen Woerter, fuer den Brown Corpus. Berechnen Sie dazu die 500 haeufigsten Woerter (nach stemming und Filterung von stopwords und Satzzeichen)

In [ ]:
from common.features import BagOfWords, WordListNormalizer
# vocabulary =

print('Filtering and stemming words in corpus...')
normalizer = WordListNormalizer()
category_wordlists_dict = normalizer.category_wordlists_dict(corpus=brown)
# Flatten the category word lists for computing overall word frequencies
# The * operator expands the list/iterator to function arguments
# itertools.chain concatenates all its parameters to a single list
print('Building Bag-of-Words vocabulary...')
wordlists = itertools.chain(*(iter(category_wordlists_dict.values())))
words = itertools.chain(*wordlists)

vocabulary_complete = BagOfWords.most_freq_words(words)
vocabulary = vocabulary_complete[:500]
#@DELETE_END

Berechnen Sie Bag-of-Words Repraesentationen fuer jedes Dokument des Brown Corpus. Verwenden Sie absolute Frequenzen. speichern Sie die Bag-of-Word Repraesentationen fuer jede Kategorie in einem 2-D NumPy Array. Speichern Sie den Bag-of-Words Vektor fuer jedes Dokument in einer Zeile, so dass das Array (ndarray) folgende Dimension hat:

 |Dokument_kat| X |Vokabular|

|Dokument_kat| entspricht der Anzahl Dokumente einer Kategorie.
|Vokabular| entspricht der Anzahl Woerter im Vokabular (hier 500).

Eine einfache Zuordnung von Kategorie und Bag-of-Words Matrix ist durch ein Dictionary moeglich.

Implementieren Sie die Funktion BagOfWords.category_bow_dict im Modul features.

In [ ]:
print('Building Bag-of-Words feature vector representations...')
bow = BagOfWords(vocabulary)
category_bow_dict = bow.category_bow_dict(category_wordlists_dict)

Testen Sie ihre Implementierung mit folgendem Unittest:

In [ ]:
import unittest

from utest.test_features import BagOfWordsTest

suite = unittest.TestSuite()
suite.addTest(BagOfWordsTest("test_category_bow_dict"))
runner = unittest.TextTestRunner()
runner.run(suite)

Um einen Klassifikator statistisch zu evaluieren, benoetigt man eine Trainingsstichprobe und eine Teststichprobe der Daten die klassifiziert werden sollen. Die Trainingsstichprobe benoetigt man zum Erstellen oder Trainieren des Klassifikators. Dabei werden in der Regel die Modellparameter des Klassifikators statistisch aus den Daten der Traingingsstichprobe geschaetzt. Die Klassenzugehoerigkeiten sind fuer die Beispiele aus der Trainingsstichprobe durch so genannte Klassenlabels gegeben.

Nachdem man den Klassifikator trainiert hat, interessiert man sich normalerweise dafuer wie gut er sich unter realen Bedingung verhaelt. Das heisst, dass der Klassifikator bei der Klassifikation zuvor unbekannter Daten moeglichst wenige Fehler machen soll. Dies simuliert man mit der Teststichprobe. Da auch fuer jedes Beispiel aus der Teststichprobe die Klassenzugehoerigkeit bekannt ist, kann man am Ende die Klassifikationsergebnisse mit den wahren Klassenlabels (aus der Teststichprobe) vergleichen und eine Fehlerrate angeben.

In dem gegebenen Brown Corpus ist keine Aufteilung in Trainings und Testdaten vorgegeben.

Waehlen Sie daher die ersten 80% der Dokumente UEBER ALLE KATEGORIEN als Trainingstichprobe und die letzten 20% der Dokumente UEBER ALLE KATEGORIEN als Teststichprobe.

Erklaeren Sie, warum Sie die Stichproben ueber alle Kategorien zusammenstellen MUESSEN.

**Antwort:**

Bitte beachten Sie, dass wir im Rahmen des Fachprojekts keinen Test auf unbekannten Testdaten simulieren. Wir haben ja bereits fuer die Erstellung der Vokabulars (haeufigste Woerter, siehe oben) den kompletten Datensatz verwendet. Stattdessen betrachten wir hier ein so genanntes Validierungsszenario, in dem wir die Klassifikationsleistung auf dem Brown Corpus optimieren. Die Ergebnisse lassen sich somit nur sehr bedingt auf unbekannte Daten uebertragen.

Erstellen Sie nun die NumPy Arrays train_samples, train_labels, test_samples und test_labels, so dass diese mit den estimate und classify Methoden der Klassen im classificaton Modul verwendet werden koennen. Teilen Sie die Daten wie oben angegeben zu 80% in Trainingsdaten und 20% in Testdaten auf.

Hinweis: Vollziehen Sie nach, wie die Klasse CrossValidation im evaluation Modul funktioniert. Wenn Sie moechten, koennen die Klasse zur Aufteilung der Daten verwenden.

In [ ]:
from common.evaluation import CrossValidation


Klassifizieren Sie nun alle Dokumente der Teststichprobe nach dem Prinzip des k-naechste-Nachbarn Klassifikators. Dabei wird die Distanz zwischen dem Merkmalsvektors eines Testbeispiels und allen Merkmalsvektoren aus der Trainingstichprobe berechnet. Das Klassenlabel des Testbeispiels wird dann ueber einen Mehrheitsentscheid der Klassenlabels der k aehnlichsten Merkmalsvektoren aus der Trainingsstichprobe bestimmt.

http://www5.informatik.uni-erlangen.de/fileadmin/Persons/NiemannHeinrich/klassifikation-von-mustern/m00-www.pdf (Abschnitt 4.2.7)

Bestimmen Sie die Distanzen von Testdaten zu Trainingsdaten mit cdist:
http://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html
Bestimmen Sie die k-naechsten Nachbarn auf Grundlage der zuvor berechneten Distanzen mit argsort:
http://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html
Ueberlegen Sie, welche zuvor von Ihnen implementierte Funktion Sie wiederverwenden koennen, um den Mehrheitsentscheid umzusetzen.

Implementieren Sie die Funktionen estimate und classify in der Klasse KNNClassifier im Modul classification.

Verwenden Sie die Euklidische Distanz und betrachten Sie zunaechst nur den naechsten Nachbarn (k=1).

HINWEIS: Hier ist zunaechst nur die Implementierung eines naechster Nachbar Klassifikators erforderlich. Diese soll aber in der naechsten Aufgabe zu einer Implementierung eines k-naechste Nachbarn Klassifikators erweitert werden. Beruechsichtigen Sie das in ihrer Implementierung.


In [ ]:
from common.classification import KNNClassifier

knn_classifier = KNNClassifier(k_neighbors=1, metric='euclidean')
knn_classifier.estimate(train_bow, train_labels)
knn_test_labels = knn_classifier.classify(test_bow)

Testen Sie ihre Implementierung mit folgendem Unittest:

In [ ]:
import unittest

from utest.test_classification import ClassificationTest

suite = unittest.TestSuite()
suite.addTest(ClassificationTest("test_nn"))
runner = unittest.TextTestRunner()
runner.run(suite)

Nachdem Sie mit dem KNNClassifier fuer jedes Testbeispiel ein Klassenlabel geschaetzt haben, koennen Sie dieses mit dem tatsaechlichen Klassenlabel vergleichen. Dieses koennen Sie wie bei den Traingingsdaten dem Corpus entnehmen.

Ermitteln Sie eine Gesamtfehlerrate und je eine Fehlerrate pro Kategorie. Implementieren Sie dazu die Klasse ClassificationEvaluator im evaluation Modul.

Warum ist diese Aufteilung der Daten in Training und Test problematisch? Was sagen die Ergebnisse aus?

In [ ]:
from common.evaluation import ClassificationEvaluator

classification_eval = ClassificationEvaluator(knn_test_labels, test_labels)
err, n_wrong, n_sampels = classification_eval.error_rate()
category_error_rates = classification_eval.category_error_rates()
print('Classification error rates: ( n_wrong, n_samples ) error_rate')
print('   Overall        : ( {:2} / {:2} ) {:.2f}'.format(n_wrong, n_sampels, err))
print('Class specific :')
for category, err, n_wrong, n_samples in category_error_rates:
    print('   {:15}: ( {:2} / {:2} ) {:.2f}'.format(category, n_wrong, n_samples, err))

Testen Sie ihre Implementierung mit folgendem Unittest:

In [ ]:
import unittest

from utest.test_evaluation import ClassificationEvaluatorTest
unittest.main(ClassificationEvaluatorTest(), argv=[''], exit=False)